In [16]:
from keras.models import load_model
from time import sleep
from keras.preprocessing.image import img_to_array
from keras.preprocessing import image
import cv2
import numpy as np

In [20]:
from keras.losses import MeanSquaredError

# Custom wrapper around MeanSquaredError, if needed
def mse(y_true, y_pred):
    return MeanSquaredError()(y_true, y_pred)


In [21]:
from keras.saving import register_keras_serializable

@register_keras_serializable(package='Custom', name='mse')
def mse(y_true, y_pred):
    return MeanSquaredError()(y_true, y_pred)


In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from collections import deque

# Load pre-trained models
face_classifier = cv2.CascadeClassifier('/kaggle/input/haarcascade/haarcascade_frontalface_default.xml')
age_model = load_model('age_model_100epochs.h5', custom_objects={'mse': mse})
gender_model = load_model('gender_model_50epochs.h5', custom_objects={'mse': mse})

# Gender labels
gender_labels = ['Male', 'Female']

# Video input path
cap = cv2.VideoCapture("/kaggle/input/webcamfaces/3196096-uhd_3840_2160_25fps.mp4")

# Define the codec and create VideoWriter object to save the video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4 files
output_path = 'output_video.mp4'  # Output video path
fps = int(cap.get(cv2.CAP_PROP_FPS))  # Get frames per second from the original video
frame_width = int(cap.get(3))  # Get width of the frame
frame_height = int(cap.get(4))  # Get height of the frame
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

age_predictions = deque(maxlen=50)  # Deque to store the last 50 age predictions

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Finished processing video or error reading frame.")
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 255, 255), 2)
        roi_color = frame[y:y+h, x:x+w]
        roi_color = cv2.resize(roi_color, (200, 200), interpolation=cv2.INTER_AREA)

        # Gender prediction
        gender_predict = gender_model.predict(np.array(roi_color).reshape(-1, 200, 200, 3), verbose=0)
        gender_predict = (gender_predict >= 0.5).astype(int)[:, 0]
        gender_label = gender_labels[gender_predict[0]]
        gender_label_position = (x, y+h+50)
        cv2.putText(frame, gender_label, gender_label_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (245, 245, 11), 2)

        # Age prediction
        age_predict = age_model.predict(np.array(roi_color).reshape(-1, 200, 200, 3), verbose=0)
        age = round(age_predict[0, 0])
        
        # Append the predicted age to the deque
        age_predictions.append(age)

        # Calculate the average of the last 50 age predictions
        if age_predictions:
            avg_age = round(np.mean(age_predictions))

        age_label_position = (x+h, y+h)
        cv2.putText(frame, "Age=" + str(avg_age), age_label_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (245, 245, 11), 2)

    # Write the frame to the output video
    out.write(frame)

cap.release()
out.release()

print(f"Video saved at {output_path}")
